In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np
import csv
import json

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [4]:
# Determining number of years in the website
html = '''
<form class="year-select" action="/vehicle-registration" accept-charset="UTF-8" method="get">
  <label for="year" aria-hidden="false" style="display: none;">Year</label>
  <select id="year" name="year" onchange="this.form.submit()" class="form-control year">
    <option selected="selected" value="2021">2021</option>
<option value="2020">2020</option>
<option value="2019">2019</option>
<option value="2018">2018</option>
<option value="2017">2017</option>
<option value="2016">2016</option>
  </select>
</form>
'''
items = soup(html, 'lxml')
options = items.find_all('option')
year =[]
for item in options:
    year.append(int(item.text.strip()))


In [7]:
counter = 1
for item in year:
    # Scrapping data for available years
    url = f'https://afdc.energy.gov/vehicle-registration?year={item}'
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, 'html.parser')
    table = html_soup.find('table', class_='table table-bordered table-striped vehicle-registration dataTable')
    car_registeration = []
    rows = table.find_all('tr')
    # Skipping the first two empty rows
    for row in rows[2:]:
        row_data = row.find_all('td')
        data_sub =[]
        for items in row_data:
            data_sub.append(items.text.strip())
        car_registeration.append(data_sub)
    car_registeration_pd = pd.DataFrame(car_registeration)
    
    car_registeration_pd.columns=['State', 'EV', 'PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
    dflist = ['EV', 'PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
#     car_registeration_pd = car_registeration_pd.set_index(car_registeration_pd.columns[0])
    # Finding the total number of cars sold in every state
    for column in dflist:
        car_registeration_pd[f'{column}']= car_registeration_pd[f'{column}'].str.replace(',', '').astype(float)
    car_registeration_pd['Total Sales'] = car_registeration_pd[dflist].sum(axis = 1)
    car_registeration_pd['EV Share'] = 100 * car_registeration_pd['EV']/car_registeration_pd['Total Sales']
    columns_to_be_removed = ['PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
    car_registeration_pd.drop(columns_to_be_removed, inplace = True, axis =1)
    car_registeration_pd.rename(columns = {'EV': f'EV_{item}', 'Total Sales': f'Total Sales_{item}', 'EV Share': f'EV Share_{item}'}, inplace = True)
#     car_registeration_pd.to_csv(f'{item}.csv')
    if counter == 1:
        car_registeration_pd_total = pd.DataFrame(car_registeration_pd)
        counter = counter + 1
    else:
        car_registeration_pd_total = pd.merge(car_registeration_pd_total, car_registeration_pd, on='State', how='inner')
car_registeration_pd_total = car_registeration_pd_total.set_index(car_registeration_pd.columns[0])
car_registeration_pd_total.head()

# car_registeration_pd_total.to_csv('data.csv')
car_registeration_pd_total.head()
    
    
    
    

,EV_2021,Total Sales_2021,EV Share_2021,EV_2020,Total Sales_2020,EV Share_2020,EV_2019,Total Sales_2019,EV Share_2019,EV_2018,Total Sales_2018,EV Share_2018,EV_2017,Total Sales_2017,EV Share_2017,EV_2016,Total Sales_2016,EV Share_2016
State,,,,,,,,,,,,,,,,,,
Alabama,4700.0,4715600.0,0.099669,2900.0,4641600.0,0.062478,2000.0,4548900.0,0.043967,1300.0,4430700.0,0.029341,800.0,4409900.0,0.018141,500.0,4382700.0,0.011408
Alaska,1300.0,562800.0,0.230988,900.0,574700.0,0.156603,700.0,587900.0,0.119068,500.0,597800.0,0.083640,400.0,619700.0,0.064547,200.0,636700.0,0.031412
Arizona,40700.0,6288700.0,0.647193,28800.0,6182200.0,0.465854,19500.0,6118700.0,0.318695,12600.0,5929600.0,0.212493,7200.0,5738000.0,0.125479,4700.0,5528100.0,0.085020
Arkansas,2400.0,2679900.0,0.089556,1300.0,2629600.0,0.049437,900.0,2578200.0,0.034908,600.0,2573400.0,0.023315,300.0,2581900.0,0.011619,200.0,2547500.0,0.007851
California,563100.0,34990100.0,1.609312,425300.0,34222800.0,1.242739,349700.0,33535000.0,1.042791,273500.0,32684700.0,0.836783,189700.0,31877400.0,0.595092,141500.0,30581200.0,0.462703


In [6]:
def make_json(csv_file_path, json_file_path):
    data = {}
    with open(csv_file_path, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        i = 1
        for rows in csvReader:
#             key = rows['State']
            data [i]=rows
            i = i+1
    with open (json_file_path, 'w', encoding = 'utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
csv_file_path = r'data.csv'
json_file_path = r'data.json'

make_json(csv_file_path, json_file_path)

In [6]:
# Quit the browsing session
browser.quit()